In [52]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
import os

In [53]:
# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('riwi_cleaner.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [54]:
class CSVCleaner:
    def __init__(self, input_file, output_file):
        self.input_file = input_file
        self.output_file = output_file
        self.estadisticas = {
            'registros_iniciales': 0,
            'duplicados_eliminados': 0,
            'nulos_eliminados': 0,
            'ventas_negativas_eliminadas': 0,
            'clientes_sin_region': 0,
            'fechas_invalidas_eliminadas': 0,
            'registros_finales': 0
        }
        
    def validar_archivo(self):
        """Validar que el archivo existe"""
        if not os.path.exists(self.input_file):
            logger.error(f"❌ Archivo no encontrado: {self.input_file}")
            return False
        
        logger.info(f"📁 Archivo encontrado: {self.input_file}")
        return True
    
    def cargar_csv(self):
        """Cargar CSV con manejo de memoria"""
        try:
            logger.info("📖 Cargando archivo CSV completo...")
            df = pd.read_csv(self.input_file)
            
            # Mapear nombres de columnas a estándar
            columnas_originales = df.columns.tolist()
            logger.info(f"📊 Columnas originales: {columnas_originales}")
            
            # Renombrar columnas a formato estándar
            mapeo_columnas = {
                'Fecha': 'fecha',
                'Producto': 'producto',
                'Tipo_Producto': 'tipo_producto',
                'Cantidad': 'cantidad',
                'Precio_Unitario': 'precio_unitario',
                'Ciudad': 'ciudad',
                'Pais': 'pais',
                'Tipo_Venta': 'tipo_venta',
                'Tipo_Cliente': 'tipo_cliente',
                'Descuento': 'descuento',
                'Costo_Envio': 'costo_envio',
                'Total': 'total'
            }
            
            # Renombrar solo las columnas que existen
            columnas_renombrar = {k: v for k, v in mapeo_columnas.items() if k in df.columns}
            df = df.rename(columns=columnas_renombrar)
            
            self.estadisticas['registros_iniciales'] = len(df)
            logger.info(f"✅ CSV cargado: {len(df)} registros")
            logger.info(f"📋 Columnas renombradas: {list(df.columns)}")
            
            return df
            
        except Exception as e:
            logger.error(f"❌ Error cargando CSV: {e}")
            return None
    
    def limpiar_duplicados(self, df):
        """Eliminar registros duplicados"""
        logger.info("🔄 Eliminando duplicados...")
        
        filas_iniciales = len(df)
        
        # Definir columnas clave para detectar duplicados (usando nombres estándar)
        columnas_clave = ['fecha', 'producto', 'cantidad', 'precio_unitario', 'ciudad', 'pais']
        
        # Verificar que todas las columnas clave existen
        columnas_existentes = [col for col in columnas_clave if col in df.columns]
        logger.info(f"📍 Columnas clave para duplicados: {columnas_existentes}")
        
        if not columnas_existentes:
            logger.warning("⚠️ No hay columnas clave válidas para detectar duplicados")
            return df
        
        # Eliminar duplicados
        df_limpio = df.drop_duplicates(subset=columnas_existentes, keep='first')
        
        duplicados_eliminados = filas_iniciales - len(df_limpio)
        self.estadisticas['duplicados_eliminados'] = duplicados_eliminados
        
        logger.info(f"✅ Duplicados eliminados: {duplicados_eliminados}")
        return df_limpio
    
    def limpiar_nulos(self, df):
        """Eliminar/Manejar valores nulos"""
        logger.info("❓ Manejando valores nulos...")
        
        filas_iniciales = len(df)
        
        # Columnas críticas que no pueden ser nulas (usando nombres estándar)
        columnas_criticas = ['fecha', 'producto', 'cantidad', 'precio_unitario', 'ciudad', 'pais', 'total']
        
        # Verificar columnas que realmente existen
        columnas_existentes = [col for col in columnas_criticas if col in df.columns]
        logger.info(f"🔍 Columnas críticas verificadas: {columnas_existentes}")
        
        if columnas_existentes:
            # Eliminar filas con valores nulos en columnas críticas
            df_limpio = df.dropna(subset=columnas_existentes)
        else:
            df_limpio = df.copy()
        
        nulos_eliminados = filas_iniciales - len(df_limpio)
        self.estadisticas['nulos_eliminados'] = nulos_eliminados
        
        # Llenar valores nulos en columnas no críticas (USANDO .loc para evitar warning)
        if 'descuento' in df_limpio.columns:
            df_limpio.loc[:, 'descuento'] = df_limpio['descuento'].fillna(0)
        
        if 'costo_envio' in df_limpio.columns:
            df_limpio.loc[:, 'costo_envio'] = df_limpio['costo_envio'].fillna(0)
        
        logger.info(f"✅ Valores nulos manejados: {nulos_eliminados} filas eliminadas")
        return df_limpio
    
    def limpiar_ventas_negativas(self, df):
        """Eliminar ventas con montos negativos o cero"""
        logger.info("💰 Eliminando ventas negativas...")
        
        filas_iniciales = len(df)
        
        # Convertir columnas numéricas
        columnas_numericas = ['cantidad', 'precio_unitario', 'total']
        for col in columnas_numericas:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Eliminar ventas negativas o cero
        condiciones = []
        if 'cantidad' in df.columns:
            condiciones.append(df['cantidad'] > 0)
        if 'precio_unitario' in df.columns:
            condiciones.append(df['precio_unitario'] > 0)
        if 'total' in df.columns:
            condiciones.append(df['total'] > 0)
        
        if condiciones:
            df_limpio = df[np.logical_and.reduce(condiciones)].copy()
        else:
            df_limpio = df.copy()
        
        ventas_negativas_eliminadas = filas_iniciales - len(df_limpio)
        self.estadisticas['ventas_negativas_eliminadas'] = ventas_negativas_eliminadas
        
        logger.info(f"✅ Ventas negativas eliminadas: {ventas_negativas_eliminadas}")
        return df_limpio
    
    def limpiar_clientes_sin_region(self, df):
        """Manejar clientes sin información de región"""
        logger.info("🌍 Manejando clientes sin región...")
        
        filas_iniciales = len(df)
        
        # Columnas de ubicación
        columnas_ubicacion = ['ciudad', 'pais']
        columnas_ubi_existentes = [col for col in columnas_ubicacion if col in df.columns]
        
        logger.info(f"📍 Columnas de ubicación: {columnas_ubi_existentes}")
        
        if columnas_ubi_existentes:
            # Eliminar filas sin información de ubicación
            df_limpio = df.dropna(subset=columnas_ubi_existentes)
            
            # También eliminar valores vacíos o espacios en blanco
            for col in columnas_ubi_existentes:
                df_limpio = df_limpio[df_limpio[col].str.strip() != '']
        else:
            df_limpio = df.copy()
        
        clientes_sin_region = filas_iniciales - len(df_limpio)
        self.estadisticas['clientes_sin_region'] = clientes_sin_region
        
        logger.info(f"✅ Clientes sin región manejados: {clientes_sin_region}")
        return df_limpio
    
    def limpiar_fechas_invalidas(self, df):
        """Eliminar fechas fuera de rango o inválidas"""
        logger.info("📅 Limpiando fechas inválidas...")
        
        filas_iniciales = len(df)
        
        if 'fecha' not in df.columns:
            logger.warning("⚠️ No se encontró columna 'fecha'")
            return df
        
        # Convertir fecha a datetime
        df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
        
        # Definir rango válido (ajústalo según tu negocio)
        fecha_min = datetime(2020, 1, 1)
        fecha_max = datetime(2026, 12, 31)
        
        # Eliminar fechas nulas o fuera de rango
        df_limpio = df[
            (df['fecha'].notna()) & 
            (df['fecha'] >= fecha_min) & 
            (df['fecha'] <= fecha_max)
        ].copy()
        
        fechas_invalidas_eliminadas = filas_iniciales - len(df_limpio)
        self.estadisticas['fechas_invalidas_eliminadas'] = fechas_invalidas_eliminadas
        
        logger.info(f"✅ Fechas inválidas eliminadas: {fechas_invalidas_eliminadas}")
        return df_limpio
    
    def normalizar_textos(self, df):
        """Normalizar campos de texto"""
        logger.info("🔤 Normalizando textos...")
        
        # Productos
        if 'producto' in df.columns:
            df['producto'] = df['producto'].str.strip().str.title()
        
        if 'tipo_producto' in df.columns:
            df['tipo_producto'] = df['tipo_producto'].str.strip().str.title()
        
        # Ubicaciones
        if 'ciudad' in df.columns:
            df['ciudad'] = df['ciudad'].str.strip().str.upper()
        
        if 'pais' in df.columns:
            df['pais'] = df['pais'].str.strip().str.upper()
        
        # Tipos de venta y cliente
        if 'tipo_venta' in df.columns:
            df['tipo_venta'] = df['tipo_venta'].str.strip().str.replace('_', ' ').str.title()
        
        if 'tipo_cliente' in df.columns:
            df['tipo_cliente'] = df['tipo_cliente'].str.strip().str.replace('_', ' ').str.title()
        
        logger.info("✅ Textos normalizados")
        return df
    
    def crear_columnas_adicionales(self, df):
        """Crear columnas útiles para análisis"""
        logger.info("🧮 Creando columnas adicionales...")
        
        # Columnas temporales (si existe fecha)
        if 'fecha' in df.columns:
            df['anio'] = df['fecha'].dt.year
            df['mes'] = df['fecha'].dt.month
            df['trimestre'] = df['fecha'].dt.quarter
            df['nombre_mes'] = df['fecha'].dt.month_name()
            df['dia_semana'] = df['fecha'].dt.day_name()
        
        # Cálculos financieros (si existen las columnas necesarias)
        if all(col in df.columns for col in ['cantidad', 'precio_unitario']):
            df['subtotal'] = df['cantidad'] * df['precio_unitario']
        
        if all(col in df.columns for col in ['subtotal', 'descuento']):
            df['monto_descuento'] = df['subtotal'] * df['descuento']
        
        if all(col in df.columns for col in ['subtotal', 'monto_descuento', 'costo_envio']):
            df['total_calculado'] = df['subtotal'] - df['monto_descuento'] + df['costo_envio']
        
        logger.info("✅ Columnas adicionales creadas")
        return df
    
    def mostrar_resumen(self):
        """Mostrar resumen de la limpieza"""
        logger.info("📊 RESUMEN DE LIMPIEZA:")
        logger.info(f"  📥 Registros iniciales: {self.estadisticas['registros_iniciales']:,}")
        logger.info(f"  🔄 Duplicados eliminados: {self.estadisticas['duplicados_eliminados']:,}")
        logger.info(f"  ❌ Valores nulos eliminados: {self.estadisticas['nulos_eliminados']:,}")
        logger.info(f"  💰 Ventas negativas eliminadas: {self.estadisticas['ventas_negativas_eliminadas']:,}")
        logger.info(f"  🌍 Clientes sin región eliminados: {self.estadisticas['clientes_sin_region']:,}")
        logger.info(f"  📅 Fechas inválidas eliminadas: {self.estadisticas['fechas_invalidas_eliminadas']:,}")
        logger.info(f"  ✅ Registros finales: {self.estadisticas['registros_finales']:,}")
        
        total_eliminados = (self.estadisticas['registros_iniciales'] - self.estadisticas['registros_finales'])
        tasa_limpieza = (total_eliminados / self.estadisticas['registros_iniciales']) * 100
        logger.info(f"  📈 Tasa de limpieza: {tasa_limpieza:.1f}%")
    
    def ejecutar_limpieza(self):
        """Ejecutar pipeline completo de limpieza"""
        logger.info("=" * 60)
        logger.info("🧹 INICIANDO LIMPIEZA DE CSV")
        logger.info("=" * 60)
        
        # Validar archivo
        if not self.validar_archivo():
            return False
        
        # Cargar datos
        df = self.cargar_csv()
        if df is None:
            return False
        
        # Aplicar limpiezas en orden
        logger.info("🔄 Aplicando pipeline de limpieza...")
        
        # 1. Eliminar duplicados
        df = self.limpiar_duplicados(df)
        
        # 2. Manejar nulos
        df = self.limpiar_nulos(df)
        
        # 3. Eliminar ventas negativas
        df = self.limpiar_ventas_negativas(df)
        
        # 4. Eliminar clientes sin región
        df = self.limpiar_clientes_sin_region(df)
        
        # 5. Eliminar fechas inválidas
        df = self.limpiar_fechas_invalidas(df)
        
        # 6. Normalizar textos
        df = self.normalizar_textos(df)
        
        # 7. Crear columnas adicionales
        df = self.crear_columnas_adicionales(df)
        
        # Estadísticas finales
        self.estadisticas['registros_finales'] = len(df)
        
        # Guardar CSV limpio
        logger.info(f"💾 Guardando CSV limpio: {self.output_file}")
        df.to_csv(self.output_file, index=False, encoding='utf-8')
        
        # Mostrar resumen
        self.mostrar_resumen()
        
        logger.info("=" * 60)
        logger.info("✅ LIMPIEZA COMPLETADA")
        logger.info("=" * 60)
        
        return True

In [55]:
if __name__ == "__main__":
    # CONFIGURACIÓN - CAMBIA ESTOS VALORES
    ARCHIVO_ENTRADA = "../Assessment /Datos.csv"  # ⚠️ CAMBIA ESTO SI ES NECESARIO
    ARCHIVO_SALIDA = "riwi_datos_limpios.csv"     # ⚠️ CAMBIA SI QUIERES OTRO NOMBRE
    
    # Crear limpiador
    limpiador = CSVCleaner(ARCHIVO_ENTRADA, ARCHIVO_SALIDA)
    
    # Ejecutar limpieza
    exito = limpiador.ejecutar_limpieza()
    
    if exito:
        print(f"🎉 ¡Limpieza completada!")
        print(f"📁 Archivo limpio guardado: {ARCHIVO_SALIDA}")
        print(f"📊 Revisa el log: riwi_cleaner.log")
    else:
        print("❌ Limpieza fallida. Revisa el log.")

2026-01-05 11:56:02,522 - INFO - ============================================================
2026-01-05 11:56:02,523 - INFO - 🧹 INICIANDO LIMPIEZA DE CSV
2026-01-05 11:56:02,523 - INFO - ============================================================
2026-01-05 11:56:02,524 - INFO - 📁 Archivo encontrado: ../Assessment /Datos.csv
2026-01-05 11:56:02,524 - INFO - 📖 Cargando archivo CSV completo...
2026-01-05 11:56:04,924 - INFO - 📊 Columnas originales: ['Fecha', 'Producto', 'Tipo_Producto', 'Cantidad', 'Precio_Unitario', 'Ciudad', 'Pais', 'Tipo_Venta', 'Tipo_Cliente', 'Descuento', 'Costo_Envio', 'Total']
2026-01-05 11:56:05,091 - INFO - ✅ CSV cargado: 2000000 registros
2026-01-05 11:56:05,092 - INFO - 📋 Columnas renombradas: ['fecha', 'producto', 'tipo_producto', 'cantidad', 'precio_unitario', 'ciudad', 'pais', 'tipo_venta', 'tipo_cliente', 'descuento', 'costo_envio', 'total']
2026-01-05 11:56:05,092 - INFO - 🔄 Aplicando pipeline de limpieza...
2026-01-05 11:56:05,092 - INFO - 🔄 Eliminando

🎉 ¡Limpieza completada!
📁 Archivo limpio guardado: riwi_datos_limpios.csv
📊 Revisa el log: riwi_cleaner.log
